In [1]:
from stoneforge.io.lasparser import parse_las_file

las3_manager = parse_las_file("../stoneforge/datasets/example_las3.las")

for key, value in las3_manager.items():
    print(key)

VERSION INFORMATION
Well Information
Log_Parameter
Log_Definition
Drilling_Definition
Drilling_Data | Drilling_Definition
Core_Definition
Core_Data[1] | Core_Definition
Core_Data[2] | Core_Definition
Inclinometry_Definition
Inclinometry_Data | Inclinometry_Definition
Test_Definition
Test_Data | Test_Definition
TOPS_Definition
TOPS_Data | TOPS_Definition
Perforations_Definition
Perforations_Data | Perforations_Definition
Log_Data | Log_Definition


In [2]:
from stoneforge.io.dlisio_r import DLISAccess

dlis_manager = DLISAccess("../stoneforge/datasets/DSDP_leg_96_hole_616_96_processed_data.dlis", vis=False)
c = dlis_manager.get_header([0, 8, 3, 9, 19, 23, 26, 31, 32, 42])
data = dlis_manager.return_data()


In [3]:
data

{'FDC/CNL/GR_main': {'B59067': {'TDEP': {'unit': 'm',
    'values': array([372.4656   , 372.3132   , 372.1608   , ...,  -4.1146564,
            -4.2670565,  -4.4194565], shape=(2474,), dtype=float32)},
   'RHOB': {'unit': 'g/cm3',
    'values': array([2.0898438, 2.0898438, 2.0898438, ..., 1.1201172, 1.1132812,
           1.109375 ], shape=(2474,), dtype=float32)},
   'GR': {'unit': 'gAPI',
    'values': array([133.25     , 133.25     , 133.25     , ...,   3.765625 ,
             3.7519531,   3.7539062], shape=(2474,), dtype=float32)},
   'CALI': {'unit': 'in',
    'values': array([7.8555613, 7.8555613, 7.8555613, ..., 7.898622 , 7.9232283,
           7.898622 ], shape=(2474,), dtype=float32)},
   'NPHI': {'unit': '%',
    'values': array([ 61.279297,  61.279297,  61.279297, ...,  96.728516, 106.640625,
            99.31641 ], shape=(2474,), dtype=float32)}}},
 'FDC/CNL/GR_repeat': {'B59180': {'TDEP': {'unit': 'm',
    'values': array([264.2616   , 264.1092   , 263.9568   , ...,  -6.553